In [2]:
# Lets read the contents of the parquet file
import pandas as pd
import pyarrow.parquet as pq
from pandas import read_pickle

# Read the parquet file into a DataFrame
df = pq.read_table('episode_000000 (1).parquet').to_pandas()

In [ ]:
df.head(5)  # Display the first 5 rows of the DataFrame

observation = [height, speed x, speed y, speed z, speed yaw]
action = [vx, vy, vz, vyaw]


action = [dvx, dvy, dvz, dvyaw]
#Controleur:
#=> speedx + dvx, speedy + dvy, speedz + dvz, speedyaw + dvyaw

,observation.state,action,timestamp,annotation.human.action.task_description,task_index,annotation.human.validity,episode_index,index,next.reward,next.done
0,"[-0.01147082911843003, 0.1220796682445753, 0.0...","[-0.010770668025204974, 0.10493434391940579, 0...",0.05,0,0,1,0,0,0.0,False
1,"[-0.011332803337926542, 0.11848813172154557, 0...","[-0.007915791733041057, 0.09781807630399446, 0...",0.10,0,0,1,0,1,0.0,False
2,"[-0.010638189675811158, 0.11397619806219476, 0...","[-0.005481704636666552, 0.08977743047161195, 0...",0.15,0,0,1,0,2,0.0,False
3,"[-0.009545243084048877, 0.10868313861539242, 0...","[-0.00323207919465483, 0.0814548304796524, 0.0...",0.20,0,0,1,0,3,0.0,False
4,"[-0.008188920744950616, 0.10271300497895948, 0...","[-0.0010329576815007518, 0.07304295233936066, ...",0.25,0,0,1,0,4,0.0,False


In [ ]:
df = pd.read_pickle("data_track_hawk/drone_control/before_processing/follow_data_v6.pkl")

# get a dataframe from a list of dictionaries
df = pd.DataFrame(df)
df.shape

In [11]:
df.head(200)  # Display the first 5 rows of the DataFrame

,timestamp,drone_position,drone_velocity,drone_orientation,drone_angular_velocity,target_position,relative_position,distance,direction,action
0,1.747600e+09,"[0.0, 0.0, -0.6810683]","[0.0, 0.0, 0.01777439]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.5043983, -0.6310591, 0.74028504]","[-0.4956017, -0.6310591, 1.4213533]",1.632208,"[-0.30363888, -0.38662916, 0.87081647]","[-0.6680055, -0.85058415, 1.9157963, -268.38477]"
1,1.747600e+09,"[-0.00026966448, -0.00038078864, -0.6741122]","[-0.010565799, -0.014447567, 0.13587467]","[0.03256183, -0.044577446, -0.030318003]","[-0.4263258, 0.35692516, -0.66230583]","[0.5043983, -0.6310591, 0.74028504]","[-0.49533206, -0.6306783, 1.4143972]",1.625924,"[-0.30464652, -0.3878892, 0.8699037]","[-0.67022234, -0.85335624, 1.9137882, -264.75006]"
2,1.747600e+09,"[-0.003393845, -0.0044223033, -0.65164536]","[-0.052973907, -0.067167744, 0.29276472]","[0.04996384, -0.082836345, -0.14293881]","[-0.21569382, 0.13580999, -1.4570227]","[0.5043983, -0.6310591, 0.74028504]","[-0.49220785, -0.6266368, 1.3919303]",1.603874,"[-0.30688682, -0.39070195, 0.8678551]","[-0.675151, -0.85954434, 1.9092811, -251.24149]"
3,1.747600e+09,"[-0.011947194, -0.015138252, -0.6124998]","[-0.11146001, -0.13885853, 0.4530757]","[0.038328916, -0.10648541, -0.337865]","[-0.07138995, 0.018182658, -2.2574983]","[0.5043983, -0.6310591, 0.74028504]","[-0.48365453, -0.61592084, 1.3527849]",1.563108,"[-0.30941847, -0.394036, 0.86544544]","[-0.6807206, -0.86687917, 1.90398, -227.83394]"
4,1.747600e+09,"[-0.02634945, -0.033054598, -0.54818535]","[-0.15891674, -0.19766589, 0.8278479]","[0.007701195, -0.115181364, -0.6077274]","[0.022629632, -0.010901026, -2.8612366]","[0.5043983, -0.6310591, 0.74028504]","[-0.46925226, -0.5980045, 1.2884704]",1.495982,"[-0.31367502, -0.39974037, 0.8612872]","[-0.69008505, -0.8794288, 1.8948318, -195.40906]"
...,...,...,...,...,...,...,...,...,...,...
195,1.747600e+09,"[11.550305, 1.4279734, 1.2415076]","[0.883353, 0.39302996, 0.0003838815]","[0.0018326226, -0.0019326772, 0.41637543]","[-0.0007991291, -0.0014277946, -0.054383438]","[12.896445, 2.0423987, 1.240285]","[0.44188264, 0.18743727, -0.0012226105]",0.479994,"[0.9205998, 0.390499, -0.0025471356]","[0.9721418, 0.412362, -0.002689743, -1.8240583]"
196,1.747600e+09,"[11.64305, 1.4691236, 1.2415441]","[0.8842104, 0.3912092, 0.00019301992]","[-0.003756103, -0.0019355761, 0.4110479]","[0.00062867027, -0.049799472, -0.04714071]","[12.944224, 2.0649955, 1.240285]","[0.3974299, 0.16779909, -0.0012590885]",0.431403,"[0.9212495, 0.38896123, -0.0029185892]","[0.87434584, 0.369158, -0.0027699948, -1.3850826]"
197,1.747600e+09,"[11.73608, 1.5101697, 1.2415653]","[0.88721985, 0.39039293, 0.0002350607]","[-0.0013394845, -0.0018341797, 0.4065042]","[0.0010952671, 0.031423077, -0.03949393]","[13.0876045, 2.133222, 1.240285]","[0.44930393, 0.19177718, -0.0012803078]",0.488522,"[0.9197202, 0.39256582, -0.002620776]","[0.9884686, 0.4219098, -0.002816677, -0.369967]"
198,1.747600e+09,"[11.82936, 1.5511109, 1.2415739]","[0.8904035, 0.38986784, -0.0001730615]","[-0.0060421987, -0.0014927431, 0.40292475]","[0.0033098185, -0.04697776, -0.028758902]","[13.18377, 2.1793292, 1.240285]","[0.4531983, 0.19483952, -0.0012888908]",0.493308,"[0.91869235, 0.39496523, -0.0026127507]","[0.9970363, 0.42864695, -0.00283556, 0.37280682]"


In [2]:
import os
import re
import glob
import numpy as np
import cv2
import pandas as pd


def transform_df(df: pd.DataFrame, episode_idx: int) -> pd.DataFrame:
    """
    Transform a filtered raw DataFrame into the desired training format.

    Columns:
      - observation.state: concatenation of drone_position and drone_orientation (6 floats)
      - action: original action (4 floats)
      - timestamp: (row_index + 1) * 2
      - annotation.human.action.task_description: always 0
      - task_index: episode_index
      - annotation.human.validity: 1
      - episode_index: episode_index
      - index: row index (0-based)
      - next.reward: 0.0
      - next.done: False
    """
    n = len(df)
    # build each column
    states = [list(vel) + list(orient) + list(ang)
              for vel, orient, ang in zip(df['drone_velocity'], df['drone_orientation'], df['drone_angular_velocity'])]
    actions = [list(a) for a in df['action']]
    timestamps = [(i + 1) * 2 for i in range(n)]
    task_desc = [0] * n
    task_idx = [episode_idx] * n
    valid = [1] * n
    ep_idx = [episode_idx] * n
    idxs = list(range(n))
    rewards = [0.0] * n
    dones = [False] * n
    dones[-1] = True  # last frame is done

    # matching the info.json's features section:
    new_df = pd.DataFrame({
        'observation.state': states,
        'action': actions,
        'timestamp': timestamps,
        'annotation.human.action.task_description': task_desc,
        'task_index': task_idx,
        'annotation.human.validity': valid,
        'episode_index': ep_idx,
        'index': idxs,
        'next.reward': rewards,
        'next.done': dones,
    })
    return new_df

def write_parquet(df: pd.DataFrame, output_data_dir: str, episode_index: int) -> str:
    """
    Write DataFrame to parquet with episode index. Returns the file path.
    """
    os.makedirs(output_data_dir, exist_ok=True)
    file_name = f"episode_{episode_index:06d}.parquet"
    out_path = os.path.join(output_data_dir, file_name)
    df.to_parquet(out_path, index=False)
    return out_path


def create_video_for_cam(src_dir: str,
                         cam_index: int,
                         out_file: str,
                         dt: float = 0.5,
                         scale_max = np.iinfo(np.uint8).max) -> list:
    """
    Create an mp4 video for a specific camera index (0=RGB,1=Depth,2=Segmentation).
    Use a fixed timestep dt seconds per frame. Returns the list of frame indices.
    """
    # collect files for this cam
    prefix = f"cam{cam_index}_"
    all_files = [f for f in os.listdir(src_dir) if f.startswith(prefix)]
    # filter by allowed extensions
    allowed = {'.bmp', '.png', '.npy'}
    files = [f for f in all_files if os.path.splitext(f)[1].lower() in allowed]
    assert files, f"No files found for cam{cam_index} in {src_dir}"
    # sort by frame index
    files = sorted(files, key=lambda x: int(re.search(r"frame(\d+)", x).group(1)))
    frame_indices = []
    frames = []
    
    for fn in files:
        idx = int(re.search(r"frame(\d+)", fn).group(1))
        frame_indices.append(idx)
        ext = os.path.splitext(fn)[1].lower()
        path = os.path.join(src_dir, fn)
        
        if ext in ('.bmp', '.png'):
            img = cv2.imread(path)
            assert img is not None, f"Failed to read image {fn}"
        elif ext == '.npy':
            arr = np.log1p(np.clip(np.load(path), a_min=0, a_max=5000))

            arr_min, arr_max = float(arr.min()), float(arr.max())
            if arr_max > arr_min:
                norm = ((arr - arr_min) / (arr_max - arr_min) * scale_max)
            else:
                norm = np.zeros_like(arr)
            
            # choose dtype based on scale_max, to make it efficient
            if scale_max <= 255:
                norm_img = norm.astype(np.uint8)
            else:
                norm_img = norm.astype(np.uint16)

            #img = cv2.applyColorMap(norm_img, cv2.COLORMAP_JET)
            if norm_img.ndim == 2:
                img = cv2.cvtColor(norm_img, cv2.COLOR_GRAY2BGR)
            else:
                img = norm_img
        else:
            continue
        frames.append(img)

    # compute fps from fixed dt
    fps = 1.0 / dt
    h, w = frames[0].shape[:2]
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    writer = cv2.VideoWriter(out_file, fourcc, fps, (w, h))
    for frame in frames:
        writer.write(frame)
    writer.release()

    return frame_indices


def process_episode(episode_idx: int,
                    before_processing_dir: str,
                    output_data_dir: str,
                    output_video_dir: str,
                    dt: float = 2) -> dict:
    """
    Process a single episode version:
      - Load and filter DataFrame
      - Build videos and get frame indices per cam
      - Verify frame indices match across cams
      - Filter DataFrame by those indices
      - Write filtered DataFrame to parquet
    Returns a dict with episode_index, paths.
    """
    # locate camera_data directory
    episode_dirs = [d for d in os.listdir(before_processing_dir)
                   if os.path.isdir(os.path.join(before_processing_dir, d))
                   and re.match(fr"episode_data_e{episode_idx}_", d)]
    assert len(episode_dirs) == 1, f"Expected one camera_data dir for episode {episode_idx}, found {episode_dirs}"
    episode_dir = os.path.join(before_processing_dir, episode_dirs[0])

    # load and filter DataFrame
    pickle_file = os.path.join(episode_dir, f"follow_data.pkl")
    df = pd.DataFrame(pd.read_pickle(pickle_file))

    # prepare video output subdirectories
    images_dir = os.path.join(output_video_dir, "observation.images.rgb")
    depth_dir = os.path.join(output_video_dir, "observation.images.depth")
    seg_dir = os.path.join(output_video_dir, "observation.images.segmentation")
    for d in (images_dir, depth_dir, seg_dir):
        os.makedirs(d, exist_ok=True)

    # define output basename
    out_name = f"episode_{episode_idx:06d}.mp4"

    # create videos and collect frame indices
    rgb_path = os.path.join(images_dir, out_name)
    idx0 = create_video_for_cam(episode_dir, 0, rgb_path, dt)
    depth_path = os.path.join(depth_dir, out_name)
    idx1 = create_video_for_cam(episode_dir, 1, depth_path, dt)
    seg_path = os.path.join(seg_dir, out_name)
    idx2 = create_video_for_cam(episode_dir, 2, seg_path, dt)

    # validate frame indices match across cams
    assert idx0 == idx1 == idx2, "Frame indices differ across cameras"

    # filter DataFrame to keep only those frame indices, in order
    df = df.iloc[idx0].reset_index(drop=True)
    df = transform_df(df, episode_idx=episode_idx)

    print(df.head(5))  # Display the first 5 rows of the filtered DataFrame
    print(f"Filtered DataFrame shape: {df.shape}")

    # write filtered DataFrame to parquet
    os.makedirs(output_data_dir, exist_ok=True)
    parquet_path = write_parquet(df, output_data_dir, episode_idx)

    return {
        "episode_index": episode_idx,
        "parquet": parquet_path,
        "videos": {
            "rgb": rgb_path,
            "depth": depth_path,
            "segmentation": seg_path,
        }
    }

In [3]:
BEFORE = "data_track_hawk/before_processing"
DATA_OUT = "data_track_hawk/dataset_drone_control/data/chunk-000"
VIDEO_OUT = "data_track_hawk/dataset_drone_control/videos/chunk-000"
for i in range(0, 6):
    result = process_episode(i, BEFORE, DATA_OUT, VIDEO_OUT)
    print("Processed episode", result)

                                   observation.state  \
0  [-0.23470455, -0.30573353, -0.04231875, 0.0666...   
1  [0.06265601, 0.07886859, 0.21626657, 0.0112875...   
2  [0.028106226, 0.043671258, -0.5838261, -0.0194...   
3  [0.29830185, 0.9936367, 0.05800056, 0.03864696...   
4  [1.2330841, -0.78998166, -0.027963877, 0.08182...   

                                              action  timestamp  \
0  [0.42617872, 0.55622643, -0.074567065, -263.61...          2   
1  [-0.28168797, -0.37319124, -0.5663822, -210.64...          4   
2  [0.14818843, 0.19872326, -0.11666328, -100.60335]          6   
3   [1.4737692, -0.25660047, 0.11007726, -171.01186]          8   
4      [0.7117931, 0.10794208, 0.05497241, 89.80819]         10   

   annotation.human.action.task_description  task_index  \
0                                         0           0   
1                                         0           0   
2                                         0           0   
3                       

In [70]:
df_ = pd.read_parquet(result["parquet"])
print("Parquet DataFrame shape:", df_.shape)

df_

Parquet DataFrame shape: (906, 10)


,observation.state,action,timestamp,annotation.human.action.task_description,task_index,annotation.human.validity,episode_index,index,next.reward,next.done
0,"[-0.23470455, -0.30573353, -0.04231875, 0.0666...","[0.42617872, 0.55622643, -0.074567065, -263.61...",2,0,0,1,0,0,0.0,False
1,"[0.06265601, 0.07886859, 0.21626657, 0.0112875...","[-0.28168797, -0.37319124, -0.5663822, -210.64...",4,0,0,1,0,1,0.0,False
2,"[0.028106226, 0.043671258, -0.5838261, -0.0194...","[0.14818843, 0.19872326, -0.11666328, -100.60335]",6,0,0,1,0,2,0.0,False
3,"[0.29830185, 0.9936367, 0.05800056, 0.03864696...","[1.4737692, -0.25660047, 0.11007726, -171.01186]",8,0,0,1,0,3,0.0,False
4,"[1.2330841, -0.78998166, -0.027963877, 0.08182...","[0.7117931, 0.10794208, 0.05497241, 89.80819]",10,0,0,1,0,4,0.0,False
...,...,...,...,...,...,...,...,...,...,...
901,"[-0.83296245, 0.577854, 0.07147529, 0.00622597...","[-0.78322166, 0.5637591, 0.066125296, -1.36562]",1804,0,0,1,0,901,0.0,False
902,"[-0.7973746, 0.6312974, 0.026983649, -0.006495...","[-0.81397295, 0.7043082, 0.022164155, -3.1143684]",1806,0,0,1,0,902,0.0,False
903,"[1.7820784, 0.6154817, -0.13647984, -0.0494263...","[2.1152744, 0.60401887, -0.02784314, 21.627617]",1808,0,0,1,0,903,0.0,False
904,"[2.0772917, 0.5936558, -0.03312884, -0.0049253...","[2.1171153, 0.5975686, -0.02711092, -0.117140315]",1810,0,0,1,0,904,0.0,False


In [4]:
import cv2
cap = cv2.VideoCapture("/Users/david/Documents/visual-intelligence/track-hawk/Isaac-GR00T/demo_data/robot_sim.PickNPlace/videos/chunk-000/observation.images.ego_view/episode_000000.mp4")
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
cap.release()
print(frame_count)

416


In [8]:
import pandas as pd
import parquet

df = pd.read_parquet("/Users/david/Documents/visual-intelligence/track-hawk/Isaac-GR00T/demo_data/robot_sim.PickNPlace/data/chunk-000/episode_000001.parquet")
df

,observation.state,action,timestamp,annotation.human.action.task_description,task_index,annotation.human.validity,episode_index,index,next.reward,next.done
0,"[0.004517572919543836, 0.11594200686530208, 0....","[0.004474349363520933, 0.11336029664835977, 0....",0.050000,2,2,1,1,416,0.0,False
1,"[0.004564400753014781, 0.1154543511807536, 0.0...","[0.005664786511372687, 0.11434291382522715, 0....",0.100000,2,2,1,1,417,0.0,False
2,"[0.004773005995151439, 0.11524983004553847, 0....","[0.006764998568003762, 0.11509602510665853, 0....",0.150000,2,2,1,1,418,0.0,False
3,"[0.005182703214343001, 0.11522344197482359, 0....","[0.007768036014083779, 0.11559397124702898, 0....",0.200000,2,2,1,1,419,0.0,False
4,"[0.005728279987571387, 0.11525023944880118, 0....","[0.008710981662045446, 0.11593084610405462, 0....",0.250000,2,2,1,1,420,0.0,False
...,...,...,...,...,...,...,...,...,...,...
465,"[0.020267644572068113, 0.039842965245885355, 0...","[0.020263139134568037, 0.03979137469164173, 0....",23.299978,2,2,1,1,881,0.0,False
466,"[0.020267490068522378, 0.039841682644254306, 0...","[0.020262883505643453, 0.039789196916078316, 0...",23.349978,2,2,1,1,882,0.0,False
467,"[0.02026733515708275, 0.03984039251162614, 0.0...","[0.020262494231189858, 0.03978581271774163, 0....",23.399978,2,2,1,1,883,0.0,False
468,"[0.020267174578066798, 0.039839055839917675, 0...","[0.02026202986006658, 0.03978174554338867, 0.0...",23.449978,2,2,1,1,884,0.0,False


In [34]:
pd.read_parquet("/Users/david/Documents/visual-intelligence/track-hawk/Isaac-GR00T/demo_data/robot_sim.PickNPlace/data/chunk-000/episode_000004.parquet").iloc[-1]

observation.state                           [0.03031124621718753, 0.04797508927745646, 0.0...
action                                      [0.030238766824946136, 0.04785935317571762, 0....
timestamp                                                                           16.399984
annotation.human.action.task_description                                                    5
task_index                                                                                  5
annotation.human.validity                                                                   1
episode_index                                                                               4
index                                                                                    2095
next.reward                                                                               1.0
next.done                                                                                True
Name: 327, dtype: object

In [53]:
pd.read_parquet("/Users/david/Downloads/episode_000000.parquet").iloc[1]

observation.state                           [-0.1011219322681427, -0.8726145625114441, -1....
action                                      [0.0005113338093178932, 0.0005684322621347659,...
absolute_action                             [0.17757156789358453, 0.007055251463779663, 0....
annotation.human.action.task_description                                                    0
annotation.human.validity                                                                   1
next.reward                                                                               0.0
next.done                                                                               False
timestamp                                                                                 0.1
frame_index                                                                                 1
episode_index                                                                               0
index                                                       

In [54]:
import cv2

cap = cv2.VideoCapture("/Users/david/Documents/visual-intelligence/track-hawk/data_track_hawk/dataset_drone_control/videos/chunk-000/observation.images.depth/episode_000000.mp4")
ret, frame = cap.read()
if ret:
    print(frame.shape)  # (height, width, channels)
cap.release()

(144, 256, 3)


In [51]:
import pandas as pd
pd.read_pickle("/Users/david/Documents/visual-intelligence/track-hawk/data_track_hawk/before_processing/episode_data_e0_box/follow_data.pkl")

[{'timestamp': 1747689419.714248,
  'drone_position': array([ 0.       ,  0.       , -0.6795402], dtype=float32),
  'drone_velocity': array([0.        , 0.        , 0.01777426], dtype=float32),
  'drone_orientation': array([0., 0., 0.], dtype=float32),
  'drone_angular_velocity': array([0., 0., 0.], dtype=float32),
  'target_position': array([ 0.5043983 , -0.6310591 ,  0.74028504], dtype=float32),
  'relative_position': array([-0.4956017, -0.6310591,  1.4198253], dtype=float32),
  'distance': 1.6308771,
  'direction': array([-0.3038866, -0.3869446,  0.8705899], dtype=float32),
  'action': array([  -0.6685505,   -0.8512781,    1.9152979, -268.38477  ],
        dtype=float32)},
 {'timestamp': 1747689419.82182,
  'drone_position': array([-2.6987414e-04, -3.8108660e-04, -6.7243618e-01], dtype=float32),
  'drone_velocity': array([-0.01057411, -0.014459  ,  0.13672169], dtype=float32),
  'drone_orientation': array([ 0.03258708, -0.04461201, -0.03031743], dtype=float32),
  'drone_angular_velo